In [1]:
import numpy as np
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import math
from datetime import *
import os
import re
import calendar

In [2]:
# ruth

path_input = r'C:\Users\YeoRuth\OneDrive - GC International AG\incentive\data_input\\'
path_output = r'C:\Users\YeoRuth\OneDrive - GC International AG\incentive\data_output\\'

In [3]:
print('On it...................')

On it...................


In [4]:
# # use with data_input1 folder

# path_input = r'C:\Users\stephenl\OneDrive - GC International AG\ML\incentive\incentive_ph\data_input2\\'
# path_output = r'C:\Users\stephenl\OneDrive - GC International AG\ML\incentive\incentive_ph\data_output2\\'

In [5]:
def roundup(x):
    i, f = divmod(x, 1)
    return int(i + ((f >= 0.5) if (x > 0) else (f > 0.5)))

In [6]:
# working with directory
os.chdir(path_input)
list1 = os.listdir()

In [7]:
# read from directory
for p in list1:
    if p.startswith('Log'):
        # read customer activities file
        ca = pd.read_excel(path_input+p,header=4).drop([0,1])
        ca.columns=ca.columns.str.lower()
        ca.columns=ca.columns.str.replace(' ','_')
        ca.sales_executive = [' '.join(i.split()) for i in ca.sales_executive]
        ca.date = pd.to_datetime(ca.date, format='%Y-%m-%d')
    if p.startswith('Transfer'):
        # read transfer order
        df = pd.read_excel(path_input+p, header=4)
        df.columns=df.columns.str.lower()
        df.columns=df.columns.str.replace(' ','_')
        df.sales_order_date = pd.to_datetime(df.sales_order_date, format='%d-%m-%Y')
        df.product_name = df.product_name.str.lower()

In [8]:
# remove persons from vietnam, indonesia, Philippines and thailand

req = (df.sales_person_name!='Truong Son Le') & (df.sales_person_name!='Prisma Armaya') & \
(df.sales_person_name!='Anuchart Maithong') & (df.sales_person_name!='Victor Mallillin')
df=df[req]

req1 = (ca.sales_executive!='Truong Son Le') & (ca.sales_executive!='Prisma Armaya') & \
(ca.sales_executive!='Anuchart Maithong') & (ca.sales_executive!='Victor Mallillin')
ca=ca[req1]

In [9]:
# define last_month

def last_month():
    m = datetime.now().month-1
    if m==0:
        n=12
    else:
        n=m
    return n

last_month=last_month()

# def year():
#     x = datetime.now().year
#     m = datetime.now().month
#     if m>=1:
#         yr = x-1
#     else:
#         yr = x
#     return yr

# define year
year = datetime.now().year

In [10]:
shortmonth = lambda x: 1 if x=='Jan' else 2 if x=='Feb' else 3 if x=='Mar' else 4 if x=='Apr' else 5 if x=='May' else \
    6 if x=='Jun' else 7 if x=='Jul' else 8 if x=='Aug' else 9 if x=='Sep' else 10 if x=='Oct' else 11 if x=='Nov' else 12 \
    if x=='Dec' else 'NA'

def first_day_month(year,last_month):
    return datetime(year,last_month,1)


def last_day_month(year,last_month):
    day = calendar.monthrange(year,last_month)[1]
    return datetime(year, last_month,day)


# convert numerical month to string month
longmonth = lambda x: 'Jan' if x==1 else 'Feb' if x==2 else 'Mar' if x==3 else 'Apr' if x==4 else 'May' if x==5 else 'Jun'\
if x==6 else 'Jul' if x==7 else 'Aug' if x==8 else 'Sep' if x==9 else 'Oct' if x==10 else 'Nov' if x==11 else 'Dec' \
if x==12 else 'NA'

In [11]:
# customer activities (3rd criteria of evaluation)

def customer_activities_last_month2():
    
    # define date range for customer activities
    req1 = (ca.date >= first_day_month(year,last_month)) & (ca.date <= last_day_month(year,last_month))
    df_ca =ca[req1]
    
    vc = df_ca.groupby(['sales_executive','region']).size().reset_index(name='results')
    vc.rename(columns={'sales_executive':'sales_rep','region':'country'},inplace=True)
    
    
    # calculate value of sales #---------------
    targetset = lambda x: 80 if x=='Hoang Nhan Ly' else 80 if x=='Dinh Dung Nguyen' else 80 if x=='Tuc Dinh Vo' else 80 if \
    x=='Van Tien Bui' else 120 #--------------------------------------------
    
#     target=100
    w=0.4
    
    vc['crm compliance']='customer activities'
    vc['target']=[targetset(i) for i in vc.sales_rep] #--------------------------
    vc['weightage']=w
    vc['achv_ratio']=vc.results/vc.target*100
    

    # distribution table
    
    vc['reward']=0

    for i, w, j in zip(vc.index, vc.weightage,vc['achv_ratio']):
        if j<65:
            vc.loc[i,'reward']=0

        elif j >= 65 and j <=100:
            x = j*w
            vc.loc[i,'reward']=roundup(x)

        elif j > 100 and j <= 104:
            j=110
            x = j*w
            vc.loc[i,'reward']=roundup(x)

        elif j > 104 and j <= 109:
            j=120
            x=j*w
            vc.loc[i,'reward']=roundup(x)

        elif j > 109 and j <=119:
            j=130
            x=j*w
            vc.loc[i,'reward']=roundup(x)

        else:
            j=140
            x=j*w
            vc.loc[i,'reward']=roundup(x)
            
                
    vc['weightage']=w
                
    vc1 = vc[['crm compliance','sales_rep','target','results','achv_ratio','weightage','reward','country']]
    
    return vc1

In [12]:
# calculating customer activities and assigning to custAct

custAct =customer_activities_last_month2()

In [13]:
df1 = pd.read_excel(path_input+'master.xlsx')

df1.crm_product_name = df1.crm_product_name.astype(str)
df1.crm_product_name=[' '.join(i.split()) for i in df1.crm_product_name]
df1.sub_category=df1.sub_category.astype(str)
df1.sub_category=[' '.join(i.split()) for i in df1.sub_category]

In [14]:
def reward_last_month2(c,b,dff):
    
    # convert numerical month to string month
    longmonth = lambda x: 'Jan' if x==1 else 'Feb' if x==2 else 'Mar' if x==3 else 'Apr' if x==4 else 'May' if x==5 else 'Jun'\
    if x==6 else 'Jul' if x==7 else 'Aug' if x==8 else 'Sep' if x==9 else 'Oct' if x==10 else 'Nov' if x==11 else 'Dec' \
    if x==12 else 'NA'
    
    # read incentive strategy to link product sub category name to crm product name
#     df1 = pd.read_excel(path_input+dff, sheet_name='master')
    df1 = pd.read_excel(path_input+'master.xlsx')

    df1.crm_product_name = df1.crm_product_name.astype(str)
    df1.crm_product_name=[' '.join(i.split()) for i in df1.crm_product_name]
    df1.sub_category=df1.sub_category.astype(str)
    df1.sub_category=[' '.join(i.split()) for i in df1.sub_category]


    ph_sm=df1.copy()
    
#     now = datetime.now()
    last_month1 = longmonth(last_month)

    
    # read target and weightage
    df2 = pd.read_excel(path_input+dff, sheet_name=last_month1) # dff is name of source file

    # select country in transfer order by product
    ph=df[df.country==c]
    
    # define date range for transfer order by product
    req = (df.sales_order_date >= first_day_month(year,last_month)) & \
    (df.sales_order_date <= last_day_month(year,last_month))
    ph=ph[req]
    
    # select sales rep in the transfer order by product
    nm = b
    ph_sales = ph[ph.sales_person_name==nm]
    
    ph_sales = ph_sales.reset_index(drop=True)


    # attached sub_category to every product code (in the transfer order)
    ph_sales['sub_category']=np.nan
    for i, p in zip(ph_sales.index, ph_sales.product_name):
        for pc, sc in zip(ph_sm.crm_product_name, ph_sm.sub_category):
            if p==pc:
                ph_sales.loc[i,'sub_category']=sc
                
    ph_sales = ph_sales.reset_index(drop=True)

    # groupby sub_category for transfer order
    ph_sales1 = ph_sales.dropna(subset=['sub_category'])
    sales_gp = ph_sales1.groupby(['sales_person_name','sub_category']).agg(qty=('qty','sum')).reset_index()
    

    # transfer qty from transfer order to target sheet based on sub-category on target sheet
    df2['qty']=0
    for i, s in zip(df2.index, df2.sub_cat):
        for sc, q in zip(sales_gp.sub_category, sales_gp.qty):
            if s==sc: # if sub category is the same
                df2.loc[i,'qty']=q

            else:
                continue
                
    df2=df2.reset_index(drop=True)
    
    
    # if target is zero, achieve also must be zero - based on selected sales_rep
    for i,q,t in zip(df2.index, df2.qty, df2[nm]):
        
        if t<=0:
            df2.loc[i,'achieve']=0
        else:
            d=q/t*100
            df2.loc[i,'achieve']=roundup(d)


    # calculate reward on target sheet
    df2['reward']=0

    for i, w, j in zip(df2.index, df2.weightage,df2.achieve):
        if j<65:
            df2.loc[i,'reward']=0

        elif j >= 65 and j <=100:
            x = j*w
            df2.loc[i,'reward']=roundup(x)

        elif j >100 and j <= 104:
            j=110
            x = j*w
            df2.loc[i,'reward']=roundup(x)

        elif j >104 and j <= 109:
            j=120
            x=j*w
            df2.loc[i,'reward']=roundup(x)

        elif j > 109 and j <=119:
            j=130
            x=j*w
            df2.loc[i,'reward']=roundup(x)

        else:
            j=140
            x=j*w
            df2.loc[i,'reward']=roundup(x)

    # rename target sheet columns and assign to df3
    df2['sales_rep']=nm
    df2.rename(columns={nm:'target'},inplace=True)
    df2['country']=c

    df3=df2[['sub_cat','target','qty','achieve','weightage','reward','sales_rep','country']]
    df3.rename(columns={'achieve':'achv_ratio'},inplace=True)
    
    print(f"Country: {c} | GC Assoc: {nm}")
    
    # calculate value of sales (2nd criteria for evaluation) "Transfer Order"

    w=0.4
    v = ph_sales.value_in_base_currency.sum()
    
    # condition for $15000 as target
    if nm=='Van Tien Bui' or nm=='Tuc Dinh Vo' or nm=='Dinh Dung Nguyen' or nm=='Hoang Nhan Ly': #-----------
        target=15000
    else:
        target=20000
        
    j = v/target*100 #-------------------------
    
    # distribution table
    
    # weightage
    w=0.4

    if j<65:
        x=0

    elif j >= 65 and j <=100:
        x = j*w

    elif j > 100 and j <= 104:
        j=110
        x = j*w

    elif j > 104 and j <= 109:
        j=120
        x=j*w

    elif j > 109 and j <=119:
        j=130
        x=j*w

    else:
        j=140
        x=j*w
        
    df4 = pd.DataFrame({'crm compliance':'transfer order','sales_rep':nm,'target':target,'results': v,
                       'achv_ratio':j,'weightage':w,'reward':x, 'country':c}, index=[0])

    # df3 is speed product sales (1st criteria of evaluation)
    # df4 is total transfer order value (2nd criteria of evaluation)
    return df3, df4

## function to return last month crm speed

In [15]:
df[df.country=='Indonesia'].sales_person_name.unique()

array(['Isthy Baroroh', 'Cecep Setiyantono'], dtype=object)

In [16]:
# calaulate rewards for speed sales (1st criteria of evaluation) across all sales assoc at once

def sea_last_month2():
    c='Singapore'
    reps = df[df.country==c].sales_person_name.unique()
    dff = 'incentive_sg2.xlsx' # use this instead of 'incentive_sg.xlsx'

    b = reps[0]
    r0,v0=reward_last_month2(c,b,dff)

    b = reps[1]
    r1,v1= reward_last_month2(c,b,dff)

    b = reps[2]
    r2,v2= reward_last_month2(c,b,dff)

    sg_last_mth = pd.concat([r0,r1,r2])
    sg_last_mth_v = pd.concat([v0,v1,v2])
    
    
    
    c='Indonesia'
    reps = df[df.country==c].sales_person_name.unique()
    dff = 'incentive_in2.xlsx'

    b = reps[0]
    r0,v0=reward_last_month2(c,b,dff)

    b = reps[1]
    r1,v1= reward_last_month2(c,b,dff)

#     b = reps[2]
#     r2,v2= reward_last_month2(c,b,dff)

    in_last_mth = pd.concat([r0,r1])
    in_last_mth_v = pd.concat([v0,v1])
    
    
    
    c='Philippines'
    reps = df[df.country==c].sales_person_name.unique()
    dff = 'incentive_ph2.xlsx'

    b = reps[0]
    r0,v0=reward_last_month2(c,b,dff)

    b = reps[1]
    r1,v1= reward_last_month2(c,b,dff)

    b = reps[2]
    r2,v2= reward_last_month2(c,b,dff)

    b = reps[3]
    r3,v3= reward_last_month2(c,b,dff)

    ph_last_mth = pd.concat([r0,r1,r2,r3])
    ph_last_mth_v = pd.concat([v0,v1,v2,v3])
    
    
    
    c='Thailand'
    reps = df[df.country==c].sales_person_name.unique()
    dff = 'incentive_th2.xlsx'

    b = reps[0]
    r0,v0=reward_last_month2(c,b,dff)

    b = reps[1]
    r1,v1= reward_last_month2(c,b,dff)

    b = reps[2]
    r2,v2= reward_last_month2(c,b,dff)

#     b = reps[3]
#     r3,v3= reward_last_month2(c,b,dff)

    th_last_mth = pd.concat([r0,r1,r2])
    th_last_mth_v = pd.concat([v0,v1,v2])
    
    
    
    c='Malaysia'
    reps = df[df.country==c].sales_person_name.unique()
    dff = 'incentive_my2.xlsx'

    b = reps[0]
    r0,v0=reward_last_month2(c,b,dff)

    b = reps[1]
    r1,v1= reward_last_month2(c,b,dff)

    b = reps[2]
    r2,v2= reward_last_month2(c,b,dff)

    b = reps[3]
    r3,v3= reward_last_month2(c,b,dff)

    b = reps[4]
    r4,v4= reward_last_month2(c,b,dff)

    my_last_mth = pd.concat([r0,r1,r2,r3,r4])
    my_last_mth_v = pd.concat([v0,v1,v2,v3,v4])
    
    
    
    c='Vietnam'
    reps = df[df.country==c].sales_person_name.unique()
    dff = 'incentive_vn2.xlsx'

    b = reps[0]
    r0,v0=reward_last_month2(c,b,dff)

    b = reps[1]
    r1,v1= reward_last_month2(c,b,dff)

    b = reps[2]
    r2,v2= reward_last_month2(c,b,dff)

    b = reps[3]
    r3,v3= reward_last_month2(c,b,dff)

    vn_last_mth = pd.concat([r0,r1,r2,r3])
    vn_last_mth_v = pd.concat([v0,v1,v2,v3])
    
    
    # a is speed.  b is transfer order
    a = pd.concat([ph_last_mth,sg_last_mth, th_last_mth, in_last_mth, my_last_mth, vn_last_mth],axis=0)
    b = pd.concat([ph_last_mth_v,sg_last_mth_v, th_last_mth_v, in_last_mth_v, my_last_mth_v, vn_last_mth_v],axis=0)
    
    return a,b
    

## Data quality

In [17]:
dataQly=custAct.copy()

In [18]:
# preparing form in excel
dataQly['crm compliance']='data quality'
dataQly['weightage']=0.2
dataQly.target=10
dataQly.results=10
dataQly['achv_ratio']=dataQly.results/dataQly.target*100
dataQly['reward']=0

In [19]:
# calculate reward on customer activities sheet
dataQly['reward']=0

for i, w, j in zip(dataQly.index, dataQly.weightage,dataQly['achv_ratio']):
    if j<65:
        dataQly.loc[i,'reward']=0

    elif j >= 65 and j <=100:
        x = j*w
        dataQly.loc[i,'reward']=roundup(x)

    elif j > 100 and j <= 104:
        j=110
        x = j*w
        dataQly.loc[i,'reward']=roundup(x)

    elif j > 104 and j <= 109:
        j=120
        x=j*w
        dataQly.loc[i,'reward']=roundup(x)

    elif j > 109 and j <=119:
        j=130
        x=j*w
        dataQly.loc[i,'reward']=roundup(x)

    else:
        j=140
        x=j*w
        dataQly.loc[i,'reward']=roundup(x)

In [20]:
# function to calculate final rewards (w/o weights)
def freward(j):
    
    if j<65:
        x=0

    elif j >= 65 and j <=100:
        x = j

    elif j > 100 and j <= 104:
        j=110
        x = j

    elif j > 104 and j <= 109:
        j=120
        x=j

    elif j > 109 and j <=119:
        j=130
        x=j

    else:
        j=140
        x=j
            
    return x
    

## Consolidating speed rewards calculation

In [21]:
# creating filenames
# creating run month names
# save to local folders in the same directory

# now = datetime.now()
last_month1 = longmonth(last_month)

filename = 'speed_'+ last_month1 + '.csv'
filename2 = 'transferOrder_' + last_month1 + '.csv'

speed,transferOrder=sea_last_month2()

Country: Singapore | GC Assoc: Jade Li
Country: Singapore | GC Assoc: Jaslyn Koh
Country: Singapore | GC Assoc: Patrick Teo
Country: Indonesia | GC Assoc: Isthy Baroroh
Country: Indonesia | GC Assoc: Cecep Setiyantono
Country: Philippines | GC Assoc: Darlene Dela Fuente
Country: Philippines | GC Assoc: Jonathan Familar
Country: Philippines | GC Assoc: Randy Santos
Country: Philippines | GC Assoc: Carlo Molina
Country: Thailand | GC Assoc: Panaschakorn Nitiwut
Country: Thailand | GC Assoc: Wiparath Wongpunpong
Country: Thailand | GC Assoc: Benjawan Wisuttilat
Country: Malaysia | GC Assoc: Palanikaviyarasan Rajaretnam
Country: Malaysia | GC Assoc: Hanafi Nordin
Country: Malaysia | GC Assoc: Intan Suraya
Country: Malaysia | GC Assoc: David Ting
Country: Malaysia | GC Assoc: Audrey Ang
Country: Vietnam | GC Assoc: Hoang Nhan Ly
Country: Vietnam | GC Assoc: Dinh Dung Nguyen
Country: Vietnam | GC Assoc: Van Tien Bui
Country: Vietnam | GC Assoc: Tuc Dinh Vo


## Calculating final speed rewards with distribution table

In [22]:
# consolidating first reward and assign to final_speed
final_speed=speed.groupby(['sales_rep','country']).agg(reward1=('reward','sum')).reset_index()

In [23]:
# calculation of final rewards for speed and assign to final_speed['reward2']
final_speed['speedReward']=[i*0.5 for i in final_speed.reward1]

# remove rewards1 column
final_speed = final_speed[['sales_rep','country','speedReward']]

# Final Speed Reward

In [24]:
# set sales rep as index
final_speed1=final_speed.set_index('sales_rep')

## Final Rewards on TransferOrder, CustAct, DataQuality

In [25]:
# merge transferOrder, custAct and dataQly rewards in one datafrome
# set sales_reps as index for all 3 dataframes

transferOrder1=transferOrder.set_index('sales_rep')
custAct1=custAct.set_index('sales_rep')
dataQly1=dataQly.set_index('sales_rep')

In [26]:
# create new dataframe to merge all 3 dataframes
secReward = dataQly[['sales_rep','country']]
secReward['tranOrdReward']=0
secReward['custActReward']=0
secReward['dataQlyReward']=0
secReward['totalReward']=0

In [27]:
secReward1=secReward.set_index('sales_rep')

In [28]:
# fill in the fields in secReward

for i, n in zip(transferOrder1.index, transferOrder1.reward):
    secReward1.loc[i,'tranOrdReward']=n
for i, n in zip(custAct1.index, custAct1.reward):
    secReward1.loc[i,'custActReward']=n
for i, n in zip(dataQly1.index, dataQly1.reward):
    secReward1.loc[i,'dataQlyReward']=n

In [29]:
# add all second part of the rewards together and assign to 'totalReward' column
secReward1['totalReward']=secReward1.tranOrdReward + secReward1.custActReward + secReward1.dataQlyReward

In [30]:
# apply 50% weightage to totalReward and assign to 'tranOrd_custAct_dataQly_Reward' columns

secReward1['tranOrd_custAct_dataQly_Reward']=[int(freward(i)*0.5) for i in secReward1.totalReward]
# secReward1=secReward1.reset_index()

## Final Reward

In [31]:
# show only 'country' and 'tranOrd_custAct_dataQly_Reward' columns
finalReward = secReward1[['country', 'tranOrd_custAct_dataQly_Reward']].reset_index()

req = (finalReward.sales_rep!='Ruth Yeo')

finalReward = finalReward[req]

In [32]:
finalReward = finalReward.set_index('sales_rep')

In [33]:
# add 'speedReward' column
for i in finalReward.index:
    finalReward.loc[i,'speedReward']=final_speed1.loc[i,'speedReward']

In [34]:
finalReward['totalReward']=finalReward.tranOrd_custAct_dataQly_Reward + finalReward.speedReward

In [35]:
finalReward=finalReward.reset_index()

In [36]:
finalReward=finalReward.sort_values('totalReward', ascending=False)

# Final Result

In [37]:
# creating filenames
# creating run month names

now = datetime.now()
last_month1 = longmonth(last_month)

filename = 'speed_'+ last_month1 + '.csv'
filename2 = 'transferOrder_' + last_month1 + '.csv'
filename3 = 'custActivities_' + last_month1 + '.csv'
filename4 = 'dataQuality_' + last_month1 + '.csv'
filename5 = 'finalReward_' + last_month1 + '.csv'

# speed,transferOrder=sea_last_month2()
speed.to_csv(path_output+filename, index=False)
transferOrder.to_csv(path_output+filename2, index=False)
custAct.to_csv(path_output+filename3, index=False)
dataQly.to_csv(path_output+filename4, index=False)
finalReward.to_csv(path_output+filename5, index=False)

In [38]:
# create new df for data quality for concatenation
dataQly_sum = dataQly[['sales_rep','target','results']]
dataQly_sum.rename(columns={'target':'dataQualityTarget','results':'dataQualityResults'},inplace=True)

custAct_sum = custAct[['sales_rep','target','results']]
custAct_sum.rename(columns={'target':'custActTarget','results':'custActResults'},inplace=True)

speed_sum = speed.groupby('sales_rep').agg(speedTarget=('target','sum'),speedResults=('qty','sum')).reset_index()

transferOrder_sum = transferOrder[['sales_rep','target','results']]
transferOrder_sum.rename(columns={'target':'transferOrderTarget','results':'transferOrderResults'},inplace=True)

finalReward_sum = finalReward[['sales_rep','totalReward','country']]

In [39]:
# concatenate all def
dataQly_cusAct = dataQly_sum.merge(custAct_sum,left_on='sales_rep', right_on='sales_rep')
dataQly_cusAct_speed = dataQly_cusAct.merge(speed_sum,left_on='sales_rep', right_on='sales_rep')
dataQly_cusAct_speed_transferOrder = dataQly_cusAct_speed.merge(transferOrder_sum,left_on='sales_rep', right_on='sales_rep')
dataQly_cusAct_speed_transferOrder_totalReward = dataQly_cusAct_speed_transferOrder.merge(finalReward_sum,
                                                                                         left_on='sales_rep', right_on='sales_rep')

In [40]:
# create new column 'month' and populate with last_month
dataQly_cusAct_speed_transferOrder_totalReward['month']=last_month

In [41]:
# save file for powerbi with month for historical records
filename6 = 'incentiveReport_' + last_month1 + '.csv'
dataQly_cusAct_speed_transferOrder_totalReward.to_csv(path_output+filename6, index=False)

In [42]:
# # save file for powerbi presentation
# filename7 = 'incentive_powerbi_' + last_month1 + '.csv'
# dataQly_cusAct_speed_transferOrder_totalReward.to_csv(path_output+filename7, index=False)

In [43]:
# # analysis of speed
# speed_analysis=speed.groupby(['sales_rep','sub_cat','country']).agg(speedTarget=('target','sum'),
#                                                                     speedResults=('qty','sum')).reset_index()

# req = (speed_analysis.speedTarget>0) & (speed_analysis.speedResults>0)
# speed_analysis1 = speed_analysis[req]
# speed_analysis1['variance']=speed_analysis1.speedResults/speed_analysis1.speedTarget
# speed_analysis2=speed_analysis1.sort_values(['country','sales_rep','variance'], ascending=False)
# speed_analysis2=speed_analysis2.reset_index(drop=True)
# speed_analysis2 = speed_analysis2[['sales_rep','sub_cat','speedTarget','speedResults','variance','country']]

In [44]:
# # analysis of speed 2

# req = speed.target > 0

# speed_analysis3 = speed[req]
# speed_analysis3.rename(columns={'target':'speedTarget','qty':'speedResults'},inplace=True)
# speed_analysis3['variance']=speed_analysis3.speedResults/speed_analysis3.speedTarget
# speed_analysis3 = speed_analysis3[['sales_rep','sub_cat','speedTarget','speedResults','variance','country']]

In [45]:
# # save file for powerbi presentation
# filename8 = 'speed_analysis_powerbi_' + last_month1 + '.csv'
# speed_analysis3.to_csv(path_output+filename8)

In [46]:
# print complete

print('Request for incentive calculation completed!')

Request for incentive calculation completed!
